In [1]:
from os import environ
environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
from __future__ import print_function
import json
from pprint import pprint
import numpy as np
from functools import reduce
import re
from nltk.tokenize import word_tokenize
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, Bidirectional, Dense, Flatten, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, UpSampling1D,BatchNormalization
from keras.layers import Activation
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
import string
from keras.models import load_model

Using TensorFlow backend.


In [3]:
def normalize(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_dictionary(vocab):
    d = dict()
    with open(vocab, 'r') as f:
        lines = f.readlines()
        for l in lines:
            w = l.strip().split()
            k = w[0]
            v = []
            for i in range(1, len(w)):
                v.append(float(w[i]))
            d[k] = np.array(v, dtype=np.float32)
    return d
def tokenize_sequence(seq, max_length, dim):
    words = word_tokenize(seq.lower())
    word_vectors = list(map(lambda x: d[x] if x in d else np.zeros(dim), words))
    for i in range(len(word_vectors), max_length):
        word_vectors.append(np.zeros(dim))
    word_vectors = word_vectors[:max_length]
    return np.array(word_vectors)
def create_one_hot_answer(para, answer, answer_start, option, max_length):
    if option == "s":
        from_begin = para[0:answer_start]
    else:
        from_begin = para[0:answer_start+len(answer)]
    l = len(word_tokenize(from_begin))
    one_hot = np.zeros(max_length)
    if option == "s":
        one_hot[min(l, max_length - 1)] = 1
    else:
        one_hot[min(l - 1, max_length-1)] = 1
    return one_hot

In [4]:
d = get_dictionary("glove/glove.6B.200d.txt")

In [5]:
with open('train.json') as data_file:
    data = json.load(data_file)
ids = []
titles = dict()
contexts = dict()
questions = dict()
answers_text = dict()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
            id_ = qas[k]["id"]
            answer = qas[k]["answer"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts[id_] = context
            answers_start[id_] = answer["answer_start"]
            answers_text[id_] = answer["text"]
            questions[id_] = question

In [6]:
max_para = 664
max_q = 50
dimension = 200


In [7]:
t = [i for i in range(len(ids))]
t.sort(key=lambda x: np.random.random())
ind = t[:len(questions)]

In [8]:
paras = [tokenize_sequence(contexts[ids[ind[i]]],max_para,dimension) for i in range(len(ind))]
qns = [tokenize_sequence(questions[ids[ind[i]]],max_q,dimension) for i in range(len(ind))]
ans_starts = [create_one_hot_answer(contexts[ids[ind[i]]], answers_text[ids[ind[i]]], answers_start[ids[ind[i]]], "s", max_para) for i in range(len(ind))]
ans_ends = [create_one_hot_answer(contexts[ids[ind[i]]], answers_text[ids[ind[i]]], answers_start[ids[ind[i]]], "e", max_para) for i in range(len(ind))]


In [9]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
BATCH_SIZE = 512
EPOCHS = 10

In [47]:
sentence = layers.Input(shape=(max_para,dimension), dtype='float32')
encoded_sentence = GRU(100, return_sequences=True)(sentence)
print(encoded_sentence.shape)
question = layers.Input(shape=(max_q,dimension), dtype='float32')
encoded_question = GRU(100, return_sequences=True)(question)

(?, ?, 100)


In [48]:
merge_1 = layers.dot([encoded_sentence, encoded_question], axes = 2 )
A_Q = layers.Activation("softmax")(merge_1)
merge_2 = layers.dot([encoded_question, encoded_sentence], axes = 2 )
A_D = layers.Activation("softmax")(merge_2)
C_Q = layers.dot([A_Q, encoded_sentence], axes = 1 )
print(C_Q.shape)

(?, ?, 100)


In [49]:
C_Q = layers.concatenate([encoded_question, C_Q], axis=2)
C_D = layers.dot([A_D, C_Q], axes=1)
C_ = layers.concatenate([encoded_sentence, C_D], axis=2)
print(C_.shape)

(?, ?, 300)


In [50]:
U = Bidirectional(LSTM(50, return_sequences=True))(C_)
print(U.shape)

(?, ?, 100)


In [51]:
#Convoluion, batch norm, relu unit
def convBN_pool(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 1, padding = 'valid', strides = 2)(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = Activation('elu')(BN)

    return activation

def convBN(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 3, padding = 'same')(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = Activation('elu')(BN)

    return activation

def RU(input_layer, conv_channels, d_rate):
    # input tensor for a 3-channel 256x256 image
    x = input_layer
    # 3x3 conv with 3 output channels (same as input channels)
    y = Conv1D(conv_channels, 3, padding='same', dilation_rate = d_rate)(x)
    y = Conv1D(conv_channels, 3, padding='same', dilation_rate = d_rate)(y)
    # this returns x + y.
    z = layers.add([x, y])
    return z

In [52]:
start = convBN(U, 100)
start = RU(start, 100, 1)
start = Dropout(0.5)(start)
start = convBN_pool(start, 64)
start = RU(start, 64, 2)
start = convBN_pool(start, 128)
start = RU(start, 128, 2)
start = convBN_pool(start, 128)
start = RU(start, 128, 2)
start = convBN_pool(start, 256)
start = RU(start, 256, 2)
start = Flatten()(start)
start = Dropout(0.5)(start)
start = Dense(max_para, activation='softmax', name='output_1')(start)


end = GRU(100, return_sequences=True)(U)
end = convBN(end, 100)
end = RU(end, 100, 1)
end = Dropout(0.5)(end)
end = convBN_pool(end, 64)
end = RU(end, 64, 2)
end = convBN_pool(end, 128)
end = RU(end, 128, 2)
end = convBN_pool(end, 128)
end = RU(end, 128, 2)
end = convBN_pool(end, 256)
end = RU(end, 256, 2)
end = Flatten()(end)
end = Dropout(0.5)(end)
end = Dense(max_para, activation='softmax', name='output_2')(end)


In [53]:
model = Model([sentence, question],[start, end])
model.compile(optimizer='adam',
              loss={'output_1': 'categorical_crossentropy', 'output_2': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [17]:
x = np.array(paras)
xq = np.array(qns)
ans_s = np.array(ans_starts)
ans_e = np.array(ans_ends)

In [54]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 664, 200)      0                                            
____________________________________________________________________________________________________
input_8 (InputLayer)             (None, 50, 200)       0                                            
____________________________________________________________________________________________________
gru_10 (GRU)                     (None, 664, 100)      90300                                        
____________________________________________________________________________________________________
gru_11 (GRU)                     (None, 50, 100)       90300                                        
___________________________________________________________________________________________

In [55]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

OSError: [Errno 12] Cannot allocate memory

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=350,
          epochs=10,
          validation_split=0.2)

Training
Train on 49103 samples, validate on 12276 samples
Epoch 1/10
49103/49103 [==============================] - 1305s - loss: 8.0058 - output_1_loss: 3.9474 - output_2_loss: 4.0583 - output_1_acc: 0.0663 - output_2_acc: 0.0533 - val_loss: 8.3351 - val_output_1_loss: 4.1538 - val_output_2_loss: 4.1813 - val_output_1_acc: 0.0661 - val_output_2_acc: 0.0492
Epoch 2/10
49103/49103 [==============================] - 1257s - loss: 7.5676 - output_1_loss: 3.7280 - output_2_loss: 3.8396 - output_1_acc: 0.0885 - output_2_acc: 0.0750 - val_loss: 8.2121 - val_output_1_loss: 4.0606 - val_output_2_loss: 4.1515 - val_output_1_acc: 0.0819 - val_output_2_acc: 0.0688
Epoch 3/10
21000/49103 [===========>..................] - ETA: 611s - loss: 7.1288 - output_1_loss: 3.5006 - output_2_loss: 3.6282 - output_1_acc: 0.1139 - output_2_acc: 0.1006

In [ ]:
model.save("model")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=350,
          epochs=10,
          validation_split=0.2)

In [ ]:
model.save("model")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=350,
          epochs=10,
          validation_split=0.2)

In [ ]:
model.save("model")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=350,
          epochs=10,
          validation_split=0.15)

In [ ]:
model.save("model")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=350,
          epochs=10,
          validation_split=0.1)

In [ ]:
model.save("model")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=350,
          epochs=10,
          validation_split=0.05)

In [ ]:
model.save("model")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=300,
          epochs=10)

In [ ]:
model.save("model_full")

In [ ]:
print('Training')
model.fit([x, xq], [ans_s, ans_e],
          batch_size=300,
          epochs=10)

In [ ]:
model.save("model_full")